In [1]:
import re
import bs4
import time
import json
import requests
import pandas as pd
import scipy as scipy
import numpy as np
from bs4 import BeautifulSoup

from datetime import datetime
from multiprocessing.pool import ThreadPool


from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver import ActionChains

In [2]:

def load_soup(url):

    r = requests.get(url)
    soup = BeautifulSoup(r.content)

    return soup


In [ ]:
def get_strains():

    driver = webdriver.Chrome(ChromeDriverManager().install())
    driver.get('https://www.leafly.com/strains')

    main_soup = BeautifulSoup(driver.page_source,'html.parser')
    element = driver.find_element_by_id("tou-continue")
    element.click()

    time.sleep(1)

    page_strains = set()

    max_pages = str(main_soup.find_all('span',class_="mx-lg md:mx-xxl")[0]).split()[-1].split("<")[0]

    for i in range(1,int(max_pages) + 1):
        driver.get(f'https://www.leafly.com/strains?page={i}')
        time.sleep(5) ## If < 20 - we might not get results on certain pages
        main_soup = BeautifulSoup(driver.page_source,'html.parser')     
        counter = 0 
        for a in main_soup.find_all('a',href=True):
            if a['href'].startswith("/strains/") and not a['href'].startswith("/strains/lists"):
                page_strains.add(a['href'])
                counter += 1
        # if (counter / 2) == 0.0:
        #     failed.append(i)
        
        print(f"Got {counter / 2} strains from page {i}")



    return page_strains



In [ ]:
page_strains = get_strains()

In [3]:
def get_feelings(soup):

    ### Get feelings
    try:
        feelings = {'Feelings':[],'Negatives':[],'Helps with':[]}

        all_feelings = soup.find_all('div',{"role":'tabpanel'})
        for i, child in enumerate(all_feelings,0):
            curr = child.findAll('div',recursive=False)
            
            if i == 0:           
                for item in curr:
                    match = re.findall("[A-Za-z]+", item.text)[0]
                    feelings['Feelings'].append(match)
            if i == 1:
                for item in curr:
                    match = re.findall("[A-Za-z]+", item.text)[0]
                    feelings['Negatives'].append(match)
            if i == 2:
                for item in curr:
                    match = re.findall("[A-Za-z]+", item.text)[0]
                    feelings['Helps with'].append(match)

                ## There are MAX 5 of each

        if len(feelings['Feelings']) < 5: ## Need to understand why empty is not here
            for i in range(5-len(feelings['Feelings'])):
                feelings['Feelings'].append(None) 

        if len(feelings['Negatives']) < 5:
            for i in range(5-len(feelings['Negatives'])):
                feelings['Negatives'].append(None) 

        if len(feelings['Helps with']) < 5:
            for i in range(5-len(feelings['Helps with'])):
                feelings['Helps with'].append(None) 

        return feelings
    except Exception as e:
        print(e)
        pass


def get_strainType(soup):

    ## Strain type - Sativa, Indica or Hybrid
    try:
        strain_type = soup.find_all('span',class_='jsx-1167018388 text-xs bg-leafly-white py-sm px-sm rounded')
        if len(strain_type) > 0:
            strain_type = strain_type[0].text
        else:
            strain_type = None

        return strain_type
    except Exception as e:
        print(e)
        pass

def get_cannabinoid(soup):
    
    ## Cannabinoids
    try:
        cannabinoids = {"THC": None, "CBD": None, "CBG":None}
        cannabinoids_list = soup.find_all('span',class_='jsx-1167018388 text-xs py-sm px-sm ml-sm rounded')
        if len(cannabinoids_list) > 0:
            for i in soup.find_all('span',class_='jsx-1167018388 text-xs py-sm px-sm ml-sm rounded'):
                if i.text.startswith("THC"):
                    cannabinoids['THC'] = i.text.split()[1]

                elif i.text.startswith("CBD"):
                    cannabinoids['CBD'] = i.text.split()[1]

                elif i.text.startswith("CBG"):
                    cannabinoids['CBG'] = i.text.split()[1]
        else:
            cannabinoids['THC'] = None
            cannabinoids['CBD'] = None
            cannabinoids['CBG'] = None

        return cannabinoids
    except Exception as e:
        print(e)
        pass

def get_rating(soup):

    ## Rating 
    try:
        rating = soup.find_all('span',class_='pr-xs')
        if len(rating) > 0:
            rating = rating[0].text
        else:
            rating = None

        return rating
    except Exception as e:
        print(e)
        pass

def get_ratingNum(soup):

    ## Number of rating users
    try:
        rating_num = soup.find_all('span',class_='pl-xs')
        if len(rating_num) > 0:
            rating_users = re.findall("[0-9]+", rating_num[0].text)[0]
        else:
            rating_users = None

        return rating_users
    except Exception as e:
        print(e)
        pass

def get_name(soup):
    try:
        name = soup.find('h1',{'itemprop':"name"})
        
        if len(name)> 0:
            name = name.text
        else:
            name = None

        return name
    except Exception as e:
        print(e)
        pass


    
    
    
# print(first_soup.find_all('div',class_='jsx-2636078472 flex flex-col md:flex-row pt-xl'))



In [ ]:
### Specific testing


driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get('https://www.leafly.com/strains/mimosa')
time.sleep(2)
main_soup = BeautifulSoup(driver.page_source,'html.parser')
element = driver.find_element_by_id("tou-continue")
element.click()

time.sleep(1)

# get_linage(main_soup)

# print(get_name(main_soup))

In [4]:
def get_linage(soup):

    left_parent = soup.find_all('div',class_='lineage__left-parent')
    right_parent = soup.find_all('div',class_='lineage__right-parent')
    left_child = soup.find_all('div',class_='lineage__left-child--two-parents')
    right_child = soup.find_all('div',class_='lineage__right-child--two-parents')

    ## Handle parents (might be single parent)
    if len(left_parent) == 0 or len(right_parent) == 0:
        parent = soup.find_all('div','lineage__center-parent')
        if len(parent) > 0:
            left_parent = parent[0].text.split("parent")[0]
            right_parent = 'N/A'
        else:
            left_parent = None
            right_parent = None
    else:
        left_parent = left_parent[0].text.split("parent")[0]
        right_parent = right_parent[0].text.split("parent")[0]

    ## Handle left child (might be child of strain with single parent)
    if len(left_child) == 0:
        left_child = soup.find_all('div',class_='lineage__left-child--single-parent')
        if len(left_child) > 0:
            left_child = left_child[0].text.split("child")[0]
        else:
            left_child = None
    else:
        left_child = left_child[0].text.split("child")[0]

    ## Handle left child (might be child of strain with single parent)
    if len(right_child) == 0:
        right_child = soup.find_all('div',class_='lineage__right-child--single-parent')
        if len(right_child) > 0:
            right_child = right_child[0].text.split("child")[0]
        else:
            right_child = None
    else:
        right_child = right_child[0].text.split("child")[0]

    return left_parent,right_parent,left_child,right_child
    
    # print(f"Left parent: {left_parent}\nRight parent: {right_parent}\nLeft child: {left_child}\nLeft child: {right_child}")

In [5]:
def scrape_data(soup):
    try:
        name = get_name(soup)
        feelings = get_feelings(soup)
        strain_type =  get_strainType(soup)
        cannabinoids =  get_cannabinoid(soup)
        rating = get_rating(soup)
        rating_users = get_ratingNum(soup)
        left_parent,right_parent,left_child,right_child = get_linage(soup)
    except Exception as e:
        print(e)
        pass


    return name,feelings, strain_type, cannabinoids, rating, rating_users, left_parent,right_parent,left_child,right_child

In [ ]:
## Testing feelings distribution

# driver = webdriver.Chrome(ChromeDriverManager().install())
# driver.get('https://www.leafly.com/strains')

# main_soup = BeautifulSoup(driver.page_source,'html.parser')
# element = driver.find_element_by_id("tou-continue")
# element.click()

# time.sleep(1)

# url = 'https://www.leafly.com/strains/purple-mac'
# driver.get(url)
# time.sleep(5)
# soup = BeautifulSoup(driver.page_source,'html.parser')
# # # feelings, strain_type, cannabinoids, rating, rating_users = scrape_data(soup)

# feelings = {'Feelings':[],'Negatives':[],'Helps with':[]}

# all_feelings = soup.find_all('div',{"role":'tabpanel'})
# for i, child in enumerate(all_feelings,0):
#     curr = child.findAll('div',recursive=False)
    
#     if i == 0:           
#         for item in curr:
#             match = re.findall("[A-Za-z]+", item.text)[0]
#             feelings['Feelings'].append(match)
#     if i == 1:
#         for item in curr:
#             match = re.findall("[A-Za-z]+", item.text)[0]
#             feelings['Negatives'].append(match)
#     if i == 2:
#         for item in curr:
#             match = re.findall("[A-Za-z]+", item.text)[0]
#             feelings['Helps with'].append(match)

# print(feelings)

# for i in soup.find_all('div',{"role":'tabpanel'}):
#     # specific_tags = i.find_all('div',class_='mb-lg relative w-full')
#     children = i.findChildren('div',recursive=False)
#     for child in children:
#         print(child.text)
#     # print(specific_tags)
#     for i, tag in enumerate(specific_tags,0):
#         ## Get only the name of the attribute
#         match = re.findall("[A-Za-z]+", tag.text)

#         if i == 0:
#             feelings['Feelings'].append(match[0])

#         elif i == 1:
#             feelings['Negatives'].append(match[0])

#         elif i == 2:
#             feelings['Helps with'].append(match[0])



In [19]:
def cannaCrawler(strains):
    ## Handle First page age limit

    # driver = webdriver.Chrome(ChromeDriverManager().install())
    driver = webdriver.Chrome('/Users/daniel.ofir/.wdm/drivers/chromedriver/mac64/90.0.4430.24/chromedriver')
    driver.get('https://www.leafly.com/strains')
  
    main_soup = BeautifulSoup(driver.page_source,'html.parser')
    element = driver.find_element_by_id("tou-continue")
    element.click()

    data = []

    # for strain in strains[:5]:
    for i,strain in enumerate(strains,0):
    # for i,strain in enumerate(strains[370:],0):
        print(f"Working on strain {strain} - #{i}")
        url = 'https://www.leafly.com' + strain
        driver.get(url)
        time.sleep(1.5)
        soup = BeautifulSoup(driver.page_source,'html.parser')
        try:
            name, feelings, strain_type, cannabinoids, rating, rating_users, left_parent,right_parent,left_child,right_child = scrape_data(soup)
            data.append({'Strain Name':name, 
                        'Feelings': feelings, 
                        'Type': strain_type, 
                        'Cannabinoids': cannabinoids, 
                        'Rating': rating, 
                        'Rating Users': rating_users,
                        'Left Parent': left_parent,
                        'Right parent': right_parent,
                        'Left Child': left_child,
                        'Right Child': right_child})
        except Exception as e:
            print(e)
            pass
        # print(feelings, strain_type, cannabinoids, rating, rating_users)
    # print(data)
    # for i in data:
    #     print(i)


    
    
    return data





In [166]:
# def for_thread(strain):
#     ## Handle First page age limit

#     data = []

#     # driver = webdriver.Chrome(ChromeDriverManager().install())
#     driver = webdriver.Chrome('/Users/daniel.ofir/.wdm/drivers/chromedriver/mac64/90.0.4430.24/chromedriver')
#     driver.get('https://www.leafly.com/strains')
    
#     time.sleep(1)
#     # main_soup = BeautifulSoup(driver.page_source,'html.parser')
#     element = driver.find_element_by_id("tou-continue")
#     element.click()

    

    

#     # for strain in strains[:5]:
#     # for i,strain in enumerate(strains,0):
#     # for i,strain in enumerate(strains[370:],0):
#     # print(f"Working on strain {strain} - #{i}")
    
#     url = 'https://www.leafly.com/' + strain
#     driver.get(url)
    
#     soup = BeautifulSoup(driver.page_source,'html.parser')
#     time.sleep(2)
#     name, feelings, strain_type, cannabinoids, rating, rating_users, left_parent,right_parent,left_child,right_child = scrape_data(soup)
#     print(name)
#     data.append({'Strain Name':name, 
#                     'Feelings': feelings, 
#                     'Type': strain_type, 
#                     'Cannabinoids': cannabinoids, 
#                     'Rating': rating, 
#                     'Rating Users': rating_users,
#                     'Left Parent': left_parent,
#                     'Right parent': right_parent,
#                     'Left Child': left_child,
#                     'Right Child': right_child})

#     # return name, feelings, strain_type, cannabinoids, rating, rating_users, left_parent,right_parent,left_child,right_child
#     return data

In [167]:
# def run_thread(strains):
#     data = []

#     counter = 0
#     with ThreadPool(15) as pool:
#         for result in pool.map(for_thread, strains):        
#             data.append(result)

#     return data
    

In [20]:
with open('strain_list.txt','r') as f:
    strains = json.loads(f.read())


# data = cannaCrawler(strains)
now = datetime.now()
start_time = now.strftime("%H:%M:%S")
print(f"current time is {start_time}")
data = cannaCrawler(strains)
# data = run_thread(strains)
now = datetime.now()
end_time = now.strftime("%H:%M:%S")
print(f"current time is {end_time}")
print("Done!!!!")

# keys = list(data[0].keys())

# for_contact = []
# df = pd.DataFrame(data[0],columns=keys)
# for_contact.append(df)

# for i in data[1:]:
#     curr_df = pd.DataFrame(i,columns=keys)
#     for_contact.append(curr_df)


# df = pd.concat(for_contact)
# df = df.reset_index(drop=True)

# df.to_csv('curr_df.csv',index=False)


strain /strains/noreaster - #4744
Working on strain /strains/tahoe-sap - #4745
Working on strain /strains/haole - #4746
Working on strain /strains/super-snow-dog - #4747
Working on strain /strains/dinachem - #4748
Working on strain /strains/guava-kai - #4749
Working on strain /strains/strawberry-blue - #4750
Working on strain /strains/day-tripper - #4751
Working on strain /strains/sour-og - #4752
Working on strain /strains/blueberry-hashplant - #4753
Working on strain /strains/velvet-glove-1 - #4754
Working on strain /strains/bubba-fett - #4755
Working on strain /strains/white-sumo - #4756
Working on strain /strains/starfighter - #4757
Working on strain /strains/death-star - #4758
Working on strain /strains/sour-pez - #4759
Working on strain /strains/dream-n-sour - #4760
Working on strain /strains/ice-bomb - #4761
Working on strain /strains/squirr-t - #4762
Working on strain /strains/cherry-do-si-dos - #4763
Working on strain /strains/east-coast-sour-diesel-5258013bbbf541e3a4e8fd93f1de

In [21]:
print(len(data))
# print(data[1])

5162


In [23]:
keys = list(data[0].keys())


# for_contact = []
df = pd.DataFrame(data,columns=keys)
# for_contact.append(df)

# for i in data[1:]:
#     curr_df = pd.DataFrame(i,columns=keys)
#     for_contact.append(curr_df)


# df = pd.concat(for_contact)
df = df.reset_index(drop=True)

# df.to_csv('final_df.csv',index=False)


# df = pd.DataFrame(data)

df['Feeling_1'] = df.apply(lambda x: x['Feelings']['Feelings'][0],axis=1) ## Turn feelings list to columns
df['Feeling_2'] = df.apply(lambda x: x['Feelings']['Feelings'][1],axis=1) 
df['Feeling_3'] = df.apply(lambda x: x['Feelings']['Feelings'][2],axis=1) 
df['Feeling_4'] = df.apply(lambda x: x['Feelings']['Feelings'][3],axis=1) 
df['Feeling_5'] = df.apply(lambda x: x['Feelings']['Feelings'][4],axis=1) 

df['Negative_1'] = df.apply(lambda x: x['Feelings']['Negatives'][0],axis=1) ## Turn negative feelings list to columns
df['Negative_2'] = df.apply(lambda x: x['Feelings']['Negatives'][1],axis=1) 
df['Negative_3'] = df.apply(lambda x: x['Feelings']['Negatives'][2],axis=1) 
df['Negative_4'] = df.apply(lambda x: x['Feelings']['Negatives'][3],axis=1) 
df['Negative_5'] = df.apply(lambda x: x['Feelings']['Negatives'][4],axis=1) 

df['Helps with_1'] = df.apply(lambda x: x['Feelings']['Helps with'][0],axis=1) ## Turn helps with list to columns
df['Helps with_2'] = df.apply(lambda x: x['Feelings']['Helps with'][1],axis=1) 
df['Helps with_3'] = df.apply(lambda x: x['Feelings']['Helps with'][2],axis=1) 
df['Helps with_4'] = df.apply(lambda x: x['Feelings']['Helps with'][3],axis=1) 
df['Helps with_5'] = df.apply(lambda x: x['Feelings']['Helps with'][4],axis=1) 

df['THC'] = df.apply(lambda x: x['Cannabinoids']['THC'],axis=1) 
df['CBD'] = df.apply(lambda x: x['Cannabinoids']['CBD'],axis=1) 
df['CBG'] = df.apply(lambda x: x['Cannabinoids']['CBG'],axis=1) 


df.drop(['Feelings','Cannabinoids'],inplace=True,axis=1)

# df['Feeling_1'] = df['Feelings'][0]['Feelings'][0]
# df['Feeling_2'] = df['Feelings'][0]['Feelings'][1]

# print(len(df['Feelings'][4]))
# print(len(df['Feelings'][4]['Feelings']))

# print(df.iloc[9]['Feelings'])


# print(df.columns)
# print(df.head())



# df.describe()


,Strain Name,Type,Rating,Rating Users,Left Parent,Right parent,Left Child,Right Child,Feeling_1,Feeling_2,...,Negative_4,Negative_5,Helps with_1,Helps with_2,Helps with_3,Helps with_4,Helps with_5,THC,CBD,CBG
0,Purple MAC,None,5.0,5,None,None,None,None,Happy,Euphoric,...,None,None,None,None,None,None,None,None,None,None
1,Mule Fuel,Hybrid,4.8,8,None,None,None,None,Sleepy,Relaxed,...,Headaches,Inflammation,None,None,None,None,None,27%,None,1%
2,Strawberry Guava,Hybrid,4.8,16,None,None,None,None,Relaxed,Happy,...,None,None,None,None,None,None,None,22%,None,1%
3,Gutbuster,Indica,4.7,17,Cookies and Cream,Kimbo Kush,None,None,Relaxed,Euphoric,...,Paranoid,None,Stress,Depression,Insomnia,Pain,Anxiety,18%,None,1%
4,Shawnee Hills,Hybrid,4.0,2,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [28]:
df.shape


(5162, 26)

In [ ]:

# print(df.shape)